# Cert-Elastic on Google Colab

このノートブックは、Google Drive にプロジェクトを配置して Colab 上で VS Code と同等の手順で実行できるようにしたものです。GPU を有効化してから実行してください。

- 実行対象: `run_evaluate_cert.py`（ログ計測＋要約）
- 保存先: Google Drive (`/content/drive/MyDrive/Cert-Elastic`) 配下に成果物を永続化
- モデル: 省メモリのためデフォルトで 4bit 量子化を推奨します（Colab T4 / L4 でも動作）

In [ ]:
# This file will be replaced by the notebook editor

## 1) 必要ライブラリとGPUの確認
以下で Python/Pip バージョンと GPU を確認します。

In [ ]:
!python -V
!pip -V
!nvidia-smi || echo "No NVIDIA GPU"

import torch, platform
print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
print("python:", platform.python_version())

## 2) Google Drive のマウント
Drive をマウントして、プロジェクトのルートパスを設定します。

In [ ]:
from google.colab import drive
import os, pathlib

drive.mount('/content/drive')
DRIVE_ROOT = '/content/drive/MyDrive'
PROJECT_NAME = 'Cert-Elastic'
PROJECT_DIR = f'{DRIVE_ROOT}/{PROJECT_NAME}'
print('PROJECT_DIR =', PROJECT_DIR)

# HF cache to Drive to persist models between sessions
os.environ['HF_HOME'] = f'{DRIVE_ROOT}/.cache/huggingface'
os.environ['HF_DATASETS_CACHE'] = f'{DRIVE_ROOT}/.cache/huggingface/datasets'
os.environ['TRANSFORMERS_CACHE'] = f'{DRIVE_ROOT}/.cache/huggingface/hub'
pathlib.Path(os.environ['HF_HOME']).mkdir(parents=True, exist_ok=True)

## 3) Drive 内にプロジェクト用フォルダを作成
既存ならスキップされます。出力やログの保存先も合わせて作成します。

In [ ]:
import os
from datetime import datetime

SUBDIRS = ['src', 'data', 'outputs', 'models', 'logs', 'runs']
for sd in SUBDIRS:
    os.makedirs(f'{PROJECT_DIR}/{sd}', exist_ok=True)
print('created:', [f'{PROJECT_DIR}/{sd}' for sd in SUBDIRS])

## 4) リポジトリの取得 or 手元のZIP配置
- 公開GitHub: git clone（下の例ではダミーURLなので必要に応じて変更）
- 既に PC から Drive に `Cert-Elastic` フォルダをコピー済みならこのセルはスキップしてOK

In [ ]:
# いずれかを利用（既に Drive にコピー済みなら不要）
# !git clone --depth 1 https://github.com/yourname/Cert-Elastic.git /content/Cert-Elastic
# !unzip -o /content/Cert-Elastic.zip -d /content/
# !rsync -a /content/Cert-Elastic/ "$PROJECT_DIR/"

## 5) 依存関係のインストール
`requirements.txt` と PyTorch をインストールします。Colab GPU は Linux なので `bitsandbytes` も利用可能です。

In [ ]:
%cd "$PROJECT_DIR"
!python -m pip install -U pip wheel setuptools
# Colab の CUDA に合う公式 index から torch を導入（バージョンは適宜）
!pip install --index-url https://download.pytorch.org/whl/cu124 torch torchvision torchaudio

# プロジェクトの依存
!pip install -r requirements.txt

import torch
print('torch:', torch.__version__, 'cuda available:', torch.cuda.is_available())

## 6) システム依存のインストール（必要時）
このプロジェクトでは通常不要です。必要になった場合のみ以下を利用してください。

In [ ]:
# 例:
# !apt-get update -y && apt-get install -y libgl1 libglib2.0-0 graphviz

## 7) PYTHONPATH と作業ディレクトリの設定
ノートブックからローカルパッケージ `cert_elastic` を import 可能にします。

In [ ]:
%cd "$PROJECT_DIR"
import sys, os
if PROJECT_DIR not in sys.path:
    sys.path.insert(0, PROJECT_DIR)
print(sys.path[:3])

# import テスト
try:
    import cert_elastic
    print('cert_elastic imported:', cert_elastic.__file__)
except Exception as e:
    print('import error:', e)

## 8) 環境変数・.env の設定（任意）
Hugging Face Hub のトークンなどが必要な場合に設定します。

In [ ]:
import os
# 例: os.environ['HF_TOKEN'] = 'hf_XXXXXXXXXXXXXXXX'
# from huggingface_hub import login
# login(token=os.environ['HF_TOKEN'])

## 9) データセットのダウンロード/配置（任意）
本プロジェクトではサンプルプロンプトのみなので省略可。外部データが必要になったらここで準備してください。

## 10) テスト（任意）
テストがある場合はここで実行します。現状このリポジトリにはテストはありません。

## 11) アプリ/スクリプトの実行
`run_evaluate_cert.py` を Colab 向けの低メモリ設定で実行します。

In [ ]:
%cd "$PROJECT_DIR"

# 軽量化のデフォルト（Colab向け）
MODEL_ID = 'mistralai/Mistral-7B-Instruct-v0.3'
DTYPE = 'float16'          # または 'bfloat16'
LOAD_IN_4BIT = 1           # 4bit 量子化を推奨
ATNN_IMPL = 'eager'
MAX_NEW_TOKENS = 64
EVAL_PROMPTS_N = 10
RUN_FAST = 1               # 簡易速度計測をON

!python run_evaluate_cert.py \
  --model_id {MODEL_ID} \
  --dtype {DTYPE} \
  --load_in_4bit {LOAD_IN_4BIT} \
  --attn_impl {ATNN_IMPL} \
  --max_new_tokens {MAX_NEW_TOKENS} \
  --eval_prompts_n {EVAL_PROMPTS_N} \
  --run_fast {RUN_FAST} \
  --out_dir runs

## 12) 実行結果の保存とDriveへの永続化
`runs/` 以下に生成された最新の run ディレクトリを参照し、主要ファイルを表示します。

In [ ]:
import json, glob, os
from pathlib import Path

%cd "$PROJECT_DIR"
run_dirs = sorted(glob.glob('runs/run_*'))
print('runs found:', run_dirs[-3:])
if run_dirs:
    latest = run_dirs[-1]
    print('latest run:', latest)
    for f in ['config.eval.json','results.logging.json','summary.json','cert_checks.csv']:
        p = Path(latest)/f
        if p.exists():
            print(f'--- {p} (head) ---')
            if p.suffix == '.json':
                print(json.dumps(json.loads(p.read_text()) if p.read_text() else {}, ensure_ascii=False)[:1000])
            else:
                print('\n'.join(p.read_text().splitlines()[:10]))
        else:
            print(f'{p} not found')

## 13) セッション再開用の短縮セル
再接続時は以下のみ実行すればすぐ再開できます。

In [ ]:
from google.colab import drive
import os, sys

drive.mount('/content/drive', force_remount=False)
DRIVE_ROOT = '/content/drive/MyDrive'
PROJECT_NAME = 'Cert-Elastic'
PROJECT_DIR = f'{DRIVE_ROOT}/{PROJECT_NAME}'
%cd "$PROJECT_DIR"
if PROJECT_DIR not in sys.path:
    sys.path.insert(0, PROJECT_DIR)

# ここからすぐ 11) を実行可能